In [6]:
import pandas as pd
import sys

In [54]:
pd.set_option('display.max_column', 100)
pd.set_option('display.max_row', 100)

In [14]:
sys.path.append('../../../fastai/')
sys.path.append('../../')

In [15]:
# xgboost
from xgb_learn.learner import *
from xgb_learn.dataset import *
from eda.simple import *

In [18]:
# embeddings
from fastai.column_data import *
from fastai.structured import *

In [59]:
from pathlib import *

In [61]:
dirs = list(Path('../../../data/talking').iterdir())
dirs

[PosixPath('../../../data/talking/train_sample.csv'),
 PosixPath('../../../data/talking/test.csv'),
 PosixPath('../../../data/talking/train.csv')]

In [19]:
sample = pd.read_csv('../../../data/talking/train_sample.csv')

In [20]:
sample.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


In [23]:
sample.shape, sample.is_attributed.mean()

((100000, 8), 0.00227)

In [21]:
col_nas(sample)

[('attributed_time', 99773),
 ('ip', 0),
 ('app', 0),
 ('device', 0),
 ('os', 0),
 ('channel', 0),
 ('click_time', 0),
 ('is_attributed', 0)]

In [24]:
col_nunique(sample)

[('click_time', 80350),
 ('ip', 34857),
 ('attributed_time', 227),
 ('app', 161),
 ('channel', 161),
 ('os', 130),
 ('device', 100),
 ('is_attributed', 2)]

In [26]:
# extract time information
add_datepart(sample, 'click_time', drop=False, time=True)

In [30]:
nuniques = col_nunique(sample)

In [34]:
sample.drop([c for c, i in nuniques if i == 1], 1, inplace=True)

In [42]:
sample = sample.sort_values('click_time', ascending=True).reset_index(drop=True)

In [48]:
# 3 day data
sample.click_time.min(), sample.click_time.max()

(Timestamp('2017-11-06 16:00:00'), Timestamp('2017-11-09 15:59:51'))

In [50]:
col_nunique(sample)

[('click_time', 80350),
 ('click_timeElapsed', 80350),
 ('ip', 34857),
 ('attributed_time', 227),
 ('app', 161),
 ('channel', 161),
 ('os', 130),
 ('device', 100),
 ('click_timeMinute', 60),
 ('click_timeSecond', 60),
 ('click_timeHour', 24),
 ('click_timeDay', 4),
 ('click_timeDayofweek', 4),
 ('click_timeDayofyear', 4),
 ('is_attributed', 2)]

In [57]:
sample.groupby(['click_timeDay', 'click_timeHour'])['is_attributed'].count().reset_index()

,click_timeDay,click_timeHour,is_attributed
0,6,16,1192
1,6,17,717
2,6,18,414
3,6,19,298
4,6,20,235
5,6,21,263
6,6,22,649
7,6,23,1243
8,7,0,1939
9,7,1,1858


In [62]:
sample.shape

(100000, 15)

In [68]:
# create train-val sets
val_msk = (sample.click_timeDay == 9) & (sample.click_timeHour >= 10)

In [70]:
train_data = sample[~val_msk]
val_data = sample[val_msk]

In [72]:
train_data.shape, val_data.shape

((89231, 15), (10769, 15))

In [74]:
train_data.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time',
       'is_attributed', 'click_timeDay', 'click_timeDayofweek',
       'click_timeDayofyear', 'click_timeHour', 'click_timeMinute',
       'click_timeSecond', 'click_timeElapsed'],
      dtype='object')

In [81]:
# target encoding
def reg_mean_encoding(train, col, new_col, target, splits=10, seed=42):
    from sklearn.model_selection import KFold
    """ Computes regularize mean encoding.
        Use this to create mean encoding for training data
    Inputs:
        train: training dataframe
        col: a single column as string or list of columns to groupby 
        during mean target encoding
        new_col: name of new created column
        splits: splits to use for cv
    Returns:
        train: dataframe with new column added
    """
    # single column
    if isinstance(col, str):
        train[new_col] = 0
        for trn_idx, val_idx in KFold(splits, shuffle=True, random_state=seed).split(train):
            groups = train.iloc[trn_idx].groupby(col)[target].mean()
            train.loc[val_idx, new_col] = train[col][val_idx].map(groups)

        train[new_col].fillna(train[target].mean(), inplace=False)

    # multiple column
    elif isinstance(col, list):
        train[new_col] = 0
        for trn_idx, val_idx in KFold(splits, shuffle=True).split(train):
            groups = train.iloc[trn_idx].groupby(train[col].astype(object).sum(1))[target].mean()
            train.loc[val_idx, new_col] = train[col].astype(object).sum(1)[val_idx].map(groups)

        train[new_col].fillna(train[target].mean(), inplace=True)

    return train

In [139]:
# create random target encodes
# try alternative : time dependent
random_encoding_cols =  ['ip', 'app', 'device', 'os', 'channel', 'click_timeHour']
for c in random_encoding_cols:
    train_data = reg_mean_encoding(train_data, c, f'random_mean_encode_{c}', target='is_attributed')

/home/ec2-user/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [141]:
def reg_mean_encoding_test(test, train, col, new_col, target):
    """ Computes target enconding for test data.
        Use this to create mean encoding for valdiation and test data
        Inputs:
            train: training dataframe to compute means
            test: training dataframe to create new column
            col: a single column as string or list of columns
            new_col: name of new created column
        Returns:
            test: dataframe with new column added
    This is similar to how we do validation
    """
    # YOUR CODE HERE
    if isinstance(col, str):
        test[new_col] = test[col].map(train.groupby(col)[target].mean())
        test[new_col].fillna(train[target].mean(), inplace=True)
    # multiple column
    elif isinstance(col, list):
        test[new_col] = test[col].astype(object).sum(1).map(train.groupby(train[col].astype(object).sum(1))[target].mean())
        test[new_col].fillna(train[target].mean(), inplace=True)

    return test

In [142]:
# create random target encodes
# try alternative : time dependent
random_encoding_cols =  ['ip', 'app', 'device', 'os', 'channel', 'click_timeHour']
for c in random_encoding_cols:
    val_data = reg_mean_encoding_test(val_data, train_data, c, f'random_mean_encode_{c}', target='is_attributed')

/home/ec2-user/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/ec2-user/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [143]:
train_data.head(2)

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,click_timeDay,click_timeDayofweek,click_timeDayofyear,click_timeHour,click_timeMinute,click_timeSecond,click_timeElapsed,random_mean_encode_ip,random_mean_encode_app,random_mean_encode_device,random_mean_encode_os,random_mean_encode_channel,random_mean_encode_click_timeHour
0,48646,12,1,19,178,2017-11-06 16:00:00,NaN,0,6,0,310,16,0,0,1509984000,0.0,0.000094,0.001552,0.001555,0.0,0.0
1,93836,12,1,30,328,2017-11-06 16:00:09,NaN,0,6,0,310,16,0,9,1509984009,0.0,0.000094,0.001579,0.005597,0.0,0.0


In [144]:
val_data.head(2)

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,click_timeDay,click_timeDayofweek,click_timeDayofyear,click_timeHour,click_timeMinute,click_timeSecond,click_timeElapsed,random_mean_encode_ip,random_mean_encode_app,random_mean_encode_device,random_mean_encode_os,random_mean_encode_channel,random_mean_encode_click_timeHour
89231,81211,12,1,35,178,2017-11-09 10:00:00,NaN,0,9,3,313,10,0,0,1510221600,0.0,0.000084,0.001598,0.001224,0.00000,0.002945
89232,87635,18,1,8,107,2017-11-09 10:00:06,NaN,0,9,3,313,10,0,6,1510221606,0.0,0.000684,0.001598,0.000000,0.00026,0.002945


In [91]:
train_data.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time',
       'is_attributed', 'click_timeDay', 'click_timeDayofweek',
       'click_timeDayofyear', 'click_timeHour', 'click_timeMinute',
       'click_timeSecond', 'click_timeElapsed', 'random_mean_encode_ip',
       'random_mean_encode_app', 'random_mean_encode_device',
       'random_mean_encode_os', 'random_mean_encode_channel'],
      dtype='object')

### XGBOOST

In [145]:
usecols = ['ip', 'app', 'device', 'os', 'channel',
'click_timeDay', 'click_timeHour','random_mean_encode_ip',
'random_mean_encode_app', 'random_mean_encode_device',
'random_mean_encode_os', 'random_mean_encode_channel',
'random_mean_encode_click_timeHour']

In [146]:
trn_df = train_data[usecols]
val_df = val_data[usecols]
trn_y = train_data.is_attributed
val_y = val_data.is_attributed

In [147]:
trn_df.dtypes

ip                                     int64
app                                    int64
device                                 int64
os                                     int64
channel                                int64
click_timeDay                          int64
click_timeHour                         int64
random_mean_encode_ip                float64
random_mean_encode_app               float64
random_mean_encode_device            float64
random_mean_encode_os                float64
random_mean_encode_channel           float64
random_mean_encode_click_timeHour    float64
dtype: object

In [148]:
xgb_md = XGBModelData(trn_df, val_df, trn_y, val_y)

In [151]:
dtrain, dval, evals = xgb_md.get_train_eval_ds()

In [152]:
xgb_learner = XGBLearner(dtrain, evals, True, 0.001, 'binary:logistic', 'auc', seed=7)

In [153]:
xgb_learner.initialize()

Initialization Successful


In [154]:
xgb_learner.random_search(300, verbose=True)

Found new best score 0.968397
Found new best score 0.972044
Found new best score 0.972533
----------------------------------------
Best params and best iteration are found
----------------------------------------


In [155]:
# fit validation model
xgb_learner.fit_best_model(dtrain)

In [156]:
xgb_learner.best_params

{'objective': 'binary:logistic',
 'eta': 0.001,
 'eval_metric': 'auc',
 'seed': 20,
 'max_depth': 16,
 'min_child_weight': 3,
 'gamma': 2,
 'subsample': 0.7572923513538451,
 'colsample_bytree': 0.5523843608355201,
 'colsample_bylevel': 0.9553868996766688}

In [157]:
xgb_learner.best_model.best_iteration

79

In [158]:
sort_dict(xgb_learner.best_model.get_fscore())

[('random_mean_encode_app', 206),
 ('ip', 112),
 ('random_mean_encode_channel', 89),
 ('random_mean_encode_os', 82),
 ('random_mean_encode_device', 76),
 ('app', 69),
 ('random_mean_encode_click_timeHour', 68),
 ('channel', 60),
 ('random_mean_encode_ip', 55),
 ('os', 48),
 ('click_timeHour', 28),
 ('device', 16),
 ('click_timeDay', 7)]

In [159]:
xgb_learner.best_model.feature_names

['ip',
 'app',
 'device',
 'os',
 'channel',
 'click_timeDay',
 'click_timeHour',
 'random_mean_encode_ip',
 'random_mean_encode_app',
 'random_mean_encode_device',
 'random_mean_encode_os',
 'random_mean_encode_channel',
 'random_mean_encode_click_timeHour']

### EMBEDDING MODEL